In [2]:
# 梳理卷积神经网络
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


In [3]:
# 定义超参数 ———— 这里还是使用Mnist数据集来梳理
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练的总循环周期
batch_size = 64  #一个撮（批次）的大小，64张图片

# 训练集
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
# 构建网络
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cov1 = nn.Sequential(
            nn.Conv2d(  # 输入大小为(1, 28, 28)  ———— torch采用channelfirst策略，通道写在第一位
                in_channels=1,  # 灰度图输入通道只有1
                out_channels=16,  # 卷积核个数，即要得到的特征图的个数
                kernel_size=5,  # 卷积核尺寸
                stride=1,  # 步长
                padding=2,  # 边界扩充2层
                # {[(28 - 5) + 2 * 2] / 1 } + 1 = 28
            ),  # 输出特征图大小为 16 * 28 * 28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # 最大池化， 得到输出为 (16, 14, 14)

        )
        self.cov2 = nn.Sequential(  # 输入大小为 (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),  # 输出为 (32, 14, 14)
            nn.ReLU(),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.ReLU(),  # 这一步输出 (32, 14, 14)
            nn.MaxPool2d(kernel_size=2),  # 输出大小为 (32, 7, 7)
        )
        self.cov3 = nn.Sequential(
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.ReLU(),  # 输出为 (64, 7, 7)
        )

        self.out = nn.Linear(64 * 7 * 7, num_classes)

    def forward(self, x):
        # 前向传播
        x = self.cov1(x)
        x = self.cov2(x)
        x = self.cov3(x)
        x = x.view(x.size(0), -1)
        # x四个维度（batch_size, channel, h, w） flatten操作，结果为：(batch_size, 32 * 7 * 7)
        # 结果就是，batch的那个维度不变，-1 表示自动计算 即一共两个维度，第一个维度已经是batch_size,第二个维度自动计算出，使得其相乘结果等于原来的x的大小
        output = self.out(x)
        return output



### 准确率作为评估标准

In [5]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [6]:
# 实例化
net = CNN()
#损失函数
criterion = nn.CrossEntropyLoss()
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001) #定义优化器，普通的随机梯度下降算法

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)


        if batch_idx % 100 == 0:

            net.eval()
            val_rights = []

            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]))

当前epoch: 0 [0/60000 (0%)]	损失: 2.302941	训练集准确率: 6.25%	测试集正确率: 10.10%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.164504	训练集准确率: 78.99%	测试集正确率: 93.76%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.087098	训练集准确率: 86.97%	测试集正确率: 95.45%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.039481	训练集准确率: 89.96%	测试集正确率: 97.28%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.024373	训练集准确率: 91.75%	测试集正确率: 98.09%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.077950	训练集准确率: 92.94%	测试集正确率: 97.91%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.077837	训练集准确率: 93.72%	测试集正确率: 98.23%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.016997	训练集准确率: 94.33%	测试集正确率: 98.48%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.053614	训练集准确率: 94.82%	测试集正确率: 98.57%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.036403	训练集准确率: 95.18%	测试集正确率: 98.26%
当前epoch: 1 [0/60000 (0%)]	损失: 0.004105	训练集准确率: 100.00%	测试集正确率: 98.57%
当前epoch: 1 [6400/60000 (11%)]	损失: 0.015847	训练集准确率: 98.79%	测试集正确率: 98.72%
当前epoch: 1 [12800/60000 (21%)]	损失: 0.193936	训练集准确率: 98.55%	测试集正确率: 98.61%
当前epoch: 1 [19200/60000 (32%)]	损失: 0.131442	训练集准确率